In [ ]:
from openai import OpenAI
import pandas as pd
import os

# API 및 모델
client = OpenAI(api_key="..")
MODEL_NAME = "gpt-4o-mini"

# 경로
base_dir = './data/'
output_dir = './data/'
learning_path = base_dir + "클러스터별 샘플 10명 회원 상세 학습,진도이력.csv"
cluster_path = base_dir + "클러스터별 샘플 10명 회원 클러스터 정보.csv"
segment_path = base_dir + "클러스터별 샘플 10명 회원 세그먼트 정보.csv"
cluster_des_path = base_dir + "클러스터 해석.csv"

In [ ]:
# 전략 데이터 정의
strategy_data = {
    0: {
        "cluster_type": "저자기조절 학습자",
        "core_strategy": "소규모 성취 경험과 안정적인 학습 루틴을 제공하고, 외적 책임감을 통해 점진적으로 학습을 형성하는 전략",
        "praise_strategy": "소규모 목표라도 성취한 지표를 구체적으로 언급하며 칭찬하기; 작은 변화와 노력의 과정을 강조하여 성취감을 느끼게 하기",
        "alert_strategy": "지표 하락을 짚어주며, 구체적인 개선 과제를 짧은 기한 내 제시하기; 학습을 멈추지 않도록 최소한의 수행 기준을 안내하고 점검하기",
        "wrong_answer_guidance": "오답 중 하나만 골라 다시 풀어보게 하고, 복습 내용을 짧게 말로 표현해보게 하기; 복습을 짧고 명확한 수행 과제로 제시하고 완료 여부를 확인하기",
        "weekly_guidance": "이번 주에는 작고 명확한 목표 1~2개를 정해주고 실천 여부를 확인하기; 학습 시간을 정해두고, 그 안에서 해야 할 활동을 구체적으로 안내하기"
    },
    1: {
        "cluster_type": "전략적 자기조절 학습자",
        "core_strategy": "학습 중요성 강조와 환경적 구조를 통해 학습 루틴과 자기점검 습관을 형성하는 전략",
        "praise_strategy": "성취한 목표를 구체적으로 언급하며 칭찬하기; 노력한 결과를 강조하여 자부심을 느끼게 하기",
        "alert_strategy": "일정한 학습 루틴이 부족했던 부분을 짚어주며 꾸준함의 중요성 인식하게 하기; 학습 리듬을 되찾을 수 있도록 스스로 점검하고 함께 계획 조정하기",
        "wrong_answer_guidance": "오답을 통해 학습의 허점을 인식하고 복습의 필요성을 느끼게 하기; 자주 틀리는 개념 중심으로 정리하고 점검하기",
        "weekly_guidance": "학습 루틴을 정하고, 일과처럼 반복하며 자기점검으로 마무리하기; 실천 계획을 가시적으로 설정하고 환경을 구조화하기"
    },
    2: {
        "cluster_type": "순응형 학습자",
        "core_strategy": "외적 책임을 부여하여 학습 지속력을 높이는 전략",
        "praise_strategy": "지시한 과제를 빠짐없이 수행한 점을 구체적으로 언급하며 칭찬하기; 정해진 내용을 끝까지 해낸 책임감을 강조하여 자부심을 느끼게 하기",
        "alert_strategy": "누락된 복습 활동을 짚어주고, 언제까지 무엇을 복습할지 구체적으로 제시하기; 기한 내 완료하도록 지도하고 점검 계획 함께 안내하기",
        "wrong_answer_guidance": "오답 번호와 개념을 지정해 복습하게 하고, 간단한 확인 질문으로 점검하기; 복습을 필수 과제로 안내하며 완료 여부를 체크하기",
        "weekly_guidance": "정해진 일정과 방식에 맞춰 수행하도록 안내하고, 완료 여부를 반드시 확인하기; 체크리스트를 활용해 자기점검 습관을 형성하게 하기"
    },
    3: {
        "cluster_type": "고자기조절 학습자",
        "core_strategy": "자율성과 성취 경험을 강화하여 내적 동기를 지속적으로 유지하는 전략",
        "praise_strategy": "성취한 지표를 언급하며 긍정적인 피드백 전달하기; 학습 과정과 태도를 함께 칭찬하며 자기주도성 강화하기",
        "alert_strategy": "평소와 다른 학습 변화에 대해 스스로 인식하게 하기; 학습 리듬 회복을 위한 점검과 조정 유도하기",
        "wrong_answer_guidance": "오답을 통해 학습의 기회를 찾고 다시 도전해보게 하기; 부족한 개념에 대한 복습과 자기점검 권장하기",
        "weekly_guidance": "이번 주 학습도 스스로 계획하고 점검하는 흐름 유지하기; 성취 경험을 쌓으며 학습 동기를 이어가게 하기"
    },
    4: {
        "cluster_type": "회피형 학습자",
        "core_strategy": "호기심과 흥미 중심의 참여를 유도하고, 정서적 안정과 실수 허용 환경 속에서 학습 시작을 유도하는 전략",
        "praise_strategy": "작고 사소한 성취라도 구체적으로 언급하며 칭찬하기; 시도해본 과정 자체를 인정하고 긍정적 감정을 느끼게 하기",
        "alert_strategy": "지표 하락을 비판하지 않고, 다음엔 무엇을 가볍게 시도해볼지 제안하기; 부담 없이 다시 시작할 수 있다는 메시지를 전달하며 정서적으로 지지하기",
        "wrong_answer_guidance": "오답을 실수로 받아들이고, 틀려도 괜찮다는 메시지를 전달하기; 비슷한 문제를 재미 요소와 함께 다시 풀어보도록 유도하기",
        "weekly_guidance": "이번 주에는 학습이라기보다 재미 요소가 있는 콘텐츠를 하나 체험해보게 하기; 학습보다는 ‘시도해본 것 자체’에 초점을 맞춰 칭찬할 준비하기"
    },
    5: {
        "cluster_type": "회피형 학습자",
        "core_strategy": "호기심과 흥미 중심의 참여를 유도하고, 정서적 안정과 실수 허용 환경 속에서 학습 시작을 유도하는 전략",
        "praise_strategy": "작고 사소한 성취라도 구체적으로 언급하며 칭찬하기; 시도해본 과정 자체를 인정하고 긍정적 감정을 느끼게 하기",
        "alert_strategy": "지표 하락을 비판하지 않고, 다음엔 무엇을 가볍게 시도해볼지 제안하기; 부담 없이 다시 시작할 수 있다는 메시지를 전달하며 정서적으로 지지하기",
        "wrong_answer_guidance": "오답을 실수로 받아들이고, 틀려도 괜찮다는 메시지를 전달하기; 비슷한 문제를 재미 요소와 함께 다시 풀어보도록 유도하기",
        "weekly_guidance": "이번 주에는 학습이라기보다 재미 요소가 있는 콘텐츠를 하나 체험해보게 하기; 학습보다는 ‘시도해본 것 자체’에 초점을 맞춰 칭찬할 준비하기"
    },
    6: {
        "cluster_type": "순응형 학습자",
        "core_strategy": "외적 책임을 부여하여 학습 지속력을 높이는 전략",
        "praise_strategy": "지시한 과제를 빠짐없이 수행한 점을 구체적으로 언급하며 칭찬하기; 정해진 내용을 끝까지 해낸 책임감을 강조하여 자부심을 느끼게 하기",
        "alert_strategy": "누락된 복습 활동을 짚어주고, 언제까지 무엇을 복습할지 구체적으로 제시하기; 기한 내 완료하도록 지도하고 점검 계획 함께 안내하기",
        "wrong_answer_guidance": "오답 번호와 개념을 지정해 복습하게 하고, 간단한 확인 질문으로 점검하기; 복습을 필수 과제로 안내하며 완료 여부를 체크하기",
        "weekly_guidance": "정해진 일정과 방식에 맞춰 수행하도록 안내하고, 완료 여부를 반드시 확인하기; 체크리스트를 활용해 자기점검 습관을 형성하게 하기"
    },
    7: {
        "cluster_type": "순응형 학습자",
        "core_strategy": "외적 책임을 부여하여 학습 지속력을 높이는 전략",
        "praise_strategy": "지시한 과제를 빠짐없이 수행한 점을 구체적으로 언급하며 칭찬하기; 정해진 내용을 끝까지 해낸 책임감을 강조하여 자부심을 느끼게 하기",
        "alert_strategy": "누락된 복습 활동을 짚어주고, 언제까지 무엇을 복습할지 구체적으로 제시하기; 기한 내 완료하도록 지도하고 점검 계획 함께 안내하기",
        "wrong_answer_guidance": "오답 번호와 개념을 지정해 복습하게 하고, 간단한 확인 질문으로 점검하기; 복습을 필수 과제로 안내하며 완료 여부를 체크하기",
        "weekly_guidance": "정해진 일정과 방식에 맞춰 수행하도록 안내하고, 완료 여부를 반드시 확인하기; 체크리스트를 활용해 자기점검 습관을 형성하게 하기"
    }
}

In [ ]:
# 데이터 로드
df_learning = pd.read_csv(learning_path)

df_cluster = pd.read_csv(cluster_path)
df_segment = pd.read_csv(segment_path)

df_cluster_des = pd.read_csv(cluster_des_path)
df_cluster_des = df_cluster_des.rename(columns={'cluster_num': 'cluster'})


# 데이터 병합
df_cluster = pd.merge(df_cluster, df_cluster_des, on='cluster', how='inner')

##################### 메시지 생성날짜 지정 필요 #####################
msg_date = '2025-06-15'

In [ ]:
def split_weekly_df(df_learning, msg_date):

    msg_date = pd.to_datetime(msg_date)
    last_ref_date = msg_date - pd.Timedelta(days=7)
    
    # 날짜 컬럼이 문자열이면 datetime으로 변환
    df = df_learning.copy()
    df['bgn_ymd'] = pd.to_datetime(df['bgn_ymd'])
    df['end_ymd'] = pd.to_datetime(df['end_ymd'])
    
    # 이번주: msg_date가 범위 내에 있는 row
    mask_thisweek = (df['bgn_ymd'] <= msg_date) & (df['end_ymd'] >= msg_date)
    df_thisweek = df[mask_thisweek].copy()
    
    # 지난주: last_ref_date가 범위 내에 있는 row
    mask_lastweek = (df['bgn_ymd'] <= last_ref_date) & (df['end_ymd'] >= last_ref_date)
    df_lastweek = df[mask_lastweek].copy()
    
    return df_thisweek, df_lastweek

df_thisweek, df_lastweek = split_weekly_df(df_learning, msg_date)

In [ ]:
def get_week_row(df, target_date):
    target_date = pd.to_datetime(target_date)
    cond = (pd.to_datetime(df['bgn_ymd']) <= target_date) & (pd.to_datetime(df['end_ymd']) >= target_date)
    return df[cond]

df_this = get_week_row(df_learning, msg_date)
df_this.head(2)

# 1. cstmr_id와 cluster 컬럼만 추출
df_main = df_this[['cstmr_id', 'cluster']]

# 2. 중복 제거
df_main = df_main.drop_duplicates()

# 결과 확인
df_main.head(2)

,cstmr_id,cluster
989,28183290,0
1121,23140680,0


## Prompt 2. 지난주 지도가이드

In [ ]:
def make_prompt2(row, strategy, df_unit, course, unit):
    
    question_rows = df_unit[df_unit['answer_state'].isin([0, 4, 5, 6])] # 오답문항
    praise_rows = df_unit[df_unit['answer_state'].isin([1, 3])] # 정답문항
    none_rows = df_unit[df_unit['answer_state'].isna()] # nan 값

    # 오답 문항이 있는 경우
    if not question_rows.empty:

        question_info = "\n".join([
        f"  - Q{row_['question_no']} > {row_['lesn_name']} > {row_['coner_name']} > {row_['chap_name']} / 난이도: {row_['diff_nm']}, 문제유형: {row_['diff_area']} | 답안상태: {row_['answer_state']} | 학습시간: {row_['study_time']}"
        for _, row_ in question_rows.iterrows()
    ])
        print(question_info)
        prompt = f"""
[출력 사항]
1. 전체 오답 문항
{question_info}

2. 학생 지도 전략
    1) 전략: {strategy['wrong_answer_guidance']} 참고하여 문항 정보 기반 전략 생성하기.
    2) 오답 문항과 단원/차시/코너/목차 정보 기반 전략 생성하기.
    3) 전략은 2개만 생성하세요.
    4) 종결어미는 ~하기. 로 마무리하세요.

[주의 사항]
1. 과목별로 순서대로 출력되어야합니다. 과목 별로 오답 전략이 나와야 합니다.
    1) 전체오답문항
    2) 학생지도전략
2. 종결어미는 ~하기. 로 간단하게 생성하기.

[format]
1. 전체 오답 문항
2. 오답 지도 전략
"""
        print(prompt)
        return prompt
    
    # 오답 문항이 없으면서 정답 문항이 있는 경우
    elif not praise_rows.empty:
        
        prompt = f"""
[출력 사항]
1. 학습 정보
    1) 과목 > 단원
1. 칭찬 전략
    1) 과목:{course} > 단원:{unit}에 맞춰 {strategy['praise_strategy']} 문장을 반드시 포함하여 칭찬 전략 생성.
    2) 각 전략은 {course} 과목과 {unit} 단원에 맞게 생성해야함.
    3) 칭찬 전략을 반영한 칭찬 포인트를 종결어미를 ~하기. 라고 간단하게 마무리해야함.
    4) 칭찬을 나열하지 말고, 간단하게 포인트만 언급.
    5) ~하고, 이런식으로 연결하지 말고, 대화체 쓰지 말기. 선생님이 참고할 수 있는 전략을 한 줄로 간단히 쓰기.
    5) 전략은 1개 생성.

[주의 사항]
1. 반드시 전략: {strategy['praise_strategy']}을 참조해서 자연스럽게 생성하세요.
2. 반드시 과목:{course} > 단원:{unit} 정보가 들어가야 함.
3. ~하기. 로 간단하게 생성하기.

[format]
1. 학습 정보
2. 칭찬 전략
"""
        print(prompt)
        return prompt
    
    else:
        print("모든 것이 None이어서, Prompt 출력하지 않음")
        return None

In [9]:
responses2 = []

for idx, row in df_main.iterrows():
    cstmr_id = row['cstmr_id']
    cluster_id = row['cluster']
    strategy = strategy_data[cluster_id]

    # df_lastweek는 이미 지난주 데이터만 들어온 DataFrame
    df_cstmr = df_lastweek[df_lastweek['cstmr_id'] == row.cstmr_id]

    # 오답 문항 추출 (0,4,6)
    unit_lists = df_cstmr[['course_code', 'unit_name']].drop_duplicates().values.tolist()

    for unit_list in unit_lists:
        course, unit = unit_list
        df_unit = df_cstmr[
            (df_cstmr['course_code'] == course) &
            (df_cstmr['unit_name'] == unit)
        ]
        prompt = make_prompt2(row, strategy, df_unit, course, unit)
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": "당신은 초등학생 맞춤 학습 멘토입니다."},
                {"role": "user", "content": prompt}
            ],
            temperature=1,
            max_tokens=400
        )
        msg = response.choices[0].message.content
        responses2.append({
            'cstmr_id': row['cstmr_id'],   # 학생 ID 저장
            'msg_date': msg_date,
            'lesn_info' : unit_list,
            'response': msg
        })

## Prompt 3. 이번주 지도가이드

In [ ]:
def make_prompt3(row, strategy, df_thisweek_cstmr, df_cluster_cstmr):
    cluster_id = row['cluster']
    cluster_desc = df_cluster_cstmr['cluster_describe'].iloc[0] 

    # 차시(lesn) 정보 포함
    lesn_rows = df_thisweek_cstmr[['course_code', 'unit_name', 'lesn_name', 'coner_name', 'chap_name']].drop_duplicates()
    
    if lesn_rows.empty:
        lesn_info = "학습 정보 없음"
    else:
        output_lines = []
        for course, course_df in lesn_rows.groupby('course_code'):
            # 각 행을 하나의 문자열로 변환
            row_strs = [
                " > ".join([
                    str(row['unit_name']),
                    str(row['lesn_name']),
                    str(row['coner_name']),
                    str(row['chap_name'])
                ]) for _, row in course_df.iterrows()
            ]
            # 여러 줄을 \n으로 연결
            course_str = "\n".join(row_strs)
            # 과목명 + 해당 데이터 묶어서 한 번만 추가
            output_lines.append(f"{course}\n{course_str}")
        lesn_info = "\n".join(output_lines)

    prompt = f"""
다음은 cluster_id {cluster_id}번 학습자 그룹의 핵심 전략입니다:
\"\"\"{strategy}\"\"\"

[role]
당신은 초등학생 맞춤 온라인 학습 멘토입니다. {strategy['core_strategy']} 전략을 반영하여 전체 학습 방향, 과목별 이번 주 지도 가이드를 생각하세요.

[task]
1) 전체 학습 방향
    1. 전략: {strategy['core_strategy']}를 참고해서, 자연스럽게 만들어주세요.
    2. 절대 전략을 그대로 나열하지 마세요.
    3. ~하기. 라고 간단하게 2개 생성하세요.

2) 과목 별(course_code) 이번주 지도 가이드
    1. 과목별로 지도 가이드를 생성해야합니다. 
    2. 과목별로 {lesn_info}를 참조하세요. lesn_info에는 'unit_name', 'lesn_name’, ‘coner_name’, ‘chap_name’이 들어가있습니다.
    3. {cluster_desc}와 클러스터 가이드 전략 {strategy['core_strategy']}를 참조해서 지도 가이드를 생성하세요.
    4. 각 과목 별로 {strategy['core_strategy']}가 반영된 지도 가이드를 2개씩만 생성하세요.
    5. 과목 학습 정보, 과목 별 이번주 지도가이드를 과목 별로 나열하세요.
    6. 가이드는 ~하기. 라고 간단하게 생성하세요.
    7. 온라인에서 지도할 수 있도록, 비대면 방식의 지도여야합니다.

[format]
1. 전체 학습 방향
2. 과목 학습 정보 (과목 > 단원명)
3. 과목 별 이번주 지도 가이드
"""
    return prompt

In [11]:
responses3 = []

for idx, row in df_main.iterrows():
    cstmr_id = row['cstmr_id']
    cluster_id = row['cluster']

    strategy = strategy_data[cluster_id]
    df_thisweek_cstmr = df_thisweek[df_thisweek['cstmr_id'] == cstmr_id]
    df_cluster_cstmr = df_cluster[df_cluster['cluster'] == cluster_id]
    
    prompt = make_prompt3(row, strategy, df_thisweek_cstmr, df_cluster_cstmr)
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": "당신은 초등학생 맞춤 학습 멘토입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=1,
        max_tokens=400
    )
    msg = response.choices[0].message.content
    responses3.append({
        'cstmr_id': row['cstmr_id'],   # 학생 ID 저장
        'msg_date': msg_date,
        'response': msg
    })

## Prompt 결합

In [ ]:
import pandas as pd

# -------------------------------------------
# [0] 준비: responses2, responses3 → DataFrame
# -------------------------------------------
df_responses2 = pd.DataFrame(responses2)  # 컬럼: cstmr_id, msg_date, response
df_responses3 = pd.DataFrame(responses3)  # 컬럼: cstmr_id, msg_date, response

# -------------------------------------------
# [1] responses2: 그룹화 + 헤더
# -------------------------------------------
df2_grouped = (
    df_responses2
    .groupby(['cstmr_id', 'msg_date'], as_index=False)
    .agg({'response': lambda x: '\n\n'.join(x)})
    .rename(columns={'response': 'response2'})
)
df2_grouped['response2'] = "[지난주 학습 가이드]\n\n" + df2_grouped['response2']

# -------------------------------------------
# [2] responses3: 헤더 붙이기
# -------------------------------------------
df_responses3['response'] = "[이번주 학습 가이드]\n\n" + df_responses3['response']
df3 = df_responses3[['cstmr_id', 'msg_date', 'response']].rename(columns={'response': 'response3'})

# -------------------------------------------
# [3] responses merge
# -------------------------------------------
df_merged = pd.merge(df2_grouped, df3, on=['cstmr_id', 'msg_date'], how='inner')

# -------------------------------------------
# [4] cluster_describe 붙이기
# df_cluster: 컬럼에 cstmr_id, cluster_describe 있어야 함
# -------------------------------------------
df_merged = df_merged.merge(
    df_cluster[['cstmr_id', 'cluster_describe']], 
    on='cstmr_id', how='left'
)

# -------------------------------------------
# [5] seg_result 붙이기
# seg_result: 컬럼에 cstmr_id, 학습완료율, 출석률, 정답률, 단원평가 등급 있어야 함
# -------------------------------------------
df_merged = df_merged.merge(
    seg_result[['cstmr_id', '학습완료율', '출석률', '정답률', '단원평가']],
    on='cstmr_id', how='left'
)

# -------------------------------------------
# [6] df_segment에서 이름(cstmr_fnm) 붙이기
# df_segment: 컬럼에 cstmr_id, cstmr_fnm 있어야 함
# -------------------------------------------
df_merged = df_merged.merge(
    df_segment[['cstmr_id', 'cstmr_fnm']],
    on='cstmr_id', how='left'
)

# -------------------------------------------
# [7] 학습 지표 텍스트 구성
# -------------------------------------------
df_merged['learning_score'] = (
    "- 학습완료율: " + df_merged['학습완료율'] + "\n"
    + "- 출석률: " + df_merged['출석률'] + "\n"
    + "- 정답률: " + df_merged['정답률'] + "\n"
    + "- 단원평가: " + df_merged['단원평가']
)

# -------------------------------------------
# [8] ✅ result 최종 조합
# -------------------------------------------
df_merged['result'] = (
    "✅ " + df_merged['cstmr_fnm'] + " 학생 수업가이드\n\n"
    + "🎯 [학생 특성]\n" + df_merged['cluster_describe'] + "\n\n"
    + "📊 [학습 지표]\n" + df_merged['learning_score'] + "\n\n"
    + "🔍 [지난주 학습 가이드]\n\n" + df_merged['response2'].str.replace(r"^\[지난주 학습 가이드\]", "", regex=True).str.strip() + "\n\n"
    + "🗓️ [이번주 학습 가이드]\n\n" + df_merged['response3'].str.replace(r"^\[이번주 학습 가이드\]", "", regex=True).str.strip()
)



# -------------------------------------------
# [9] 최종 컬럼 선택
# -------------------------------------------
df_final = df_merged[['cstmr_id', 'msg_date', 'result']]

# -------------------------------------------
# [10] 필요하다면 CSV 저장
# -------------------------------------------
output_path = os.path.join(output_dir, f'msg_{msg_date}.csv')
df_final.to_csv(output_path, index=False, encoding='utf-8-sig')

In [ ]:
df_final.head(2)